In [1]:
import os
import pandas as pd
import json

In [2]:
website_info = pd.read_csv('./../data/Website - Website.csv')
website_info.tail()

,First Name,Last Name,Professional Headshot,Funny Headshot,GitHub,LinkedIn,Capstone Video,Podcast iframe,PDF Resume,Is Job Searching,Capstone (link),Personal Site,Website Portfolio,Tagline,Bio
21,William,McWilliams,x,NaN,https://github.com/WMcWilliams15,https://www.linkedin.com/in/will-mcwilliams/,https://youtu.be/h9iP8fiBVtc,"<iframe width=""100%"" height=""166"" scrolling=""n...",x,True,https://github.com/WMcWilliams15/NSS-Capstone,NaN,https://github.com/WMcWilliams15/NSS-Capstone,“It always seems impossible until it is done.”...,"Carpenter turned data analyst, who has an infi..."
22,Yasin,Hasan,x,NaN,https://github.com/yhasan47,https://www.linkedin.com/in/yasinhasan47,NaN,"<iframe width=""100%"" height=""166"" scrolling=""n...",NaN,True,https://github.com/yhasan47/nfl-combine-capstone,NaN,https://github.com/yhasan47/nfl-combine-capstone,"""I am ready to face any challenge that might b...",A Science Teacher turned into a Data Analyst.
23,UrLeaka,Test,x,x,https://github.com/unewsome,https://www.linkedin.com/in/urleaka-newsome/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Data is as data does""",This stuff is fun!' I can't believe I said tha...
24,Matt,Test,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Taylor,Test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Data does as data is""",NaN


---

Filter out the teachers / nan stuff

In [3]:
not_yet_graduated = ['Grady']

website_info = website_info[
    ~(
        (website_info['Last Name'] == 'Test') 
        | website_info['First Name'].isnull()
        | website_info['First Name'].isin(not_yet_graduated)
    )
]

In [13]:
website_info.head(20)

,firstName,lastName,reelThemIn,bio,github,linkedin,job_searching,portfolio,capstoneVideo,podcast,proImg,funImg,resume,id
0,Adam,Tsuchiyama,"""In God we trust. All others, bring data."" - W...",Whether it's determining how to incorporate th...,https://github.com/adamtsuch21,https://www.linkedin.com/in/adam-tsuchiyama/,True,https://github.com/adamtsuch21/nss_capstone,https://youtu.be/FB-Ia6q_Yyc,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/adam1.jpeg,../assets/img/resized/adam2.jpeg,../assets/resume/adam.pdf,1
1,Alan,Brunlinger,"""Any fool can make things complicated, it requ...",I have always had a passion for deciphering da...,https://github.com/abrunlinger,https://www.linkedin.com/in/alan-brunlinger/,True,https://github.com/abrunlinger/Quarantine-Life...,https://youtu.be/SDa7-7qzMC4,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/alan1.jpeg,../assets/img/resized/alan2.jpeg,../assets/resume/alan.pdf,2
2,Ann,Rumsey,Connecting the dots between people and data,Voracious curiosity compels me to study patter...,https://github.com/annrinTN,https://www.linkedin.com/in/ann-rumsey,False,https://annrintn.github.io/,https://youtu.be/RQCKfqIjiCU,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/ann1.jpeg,../assets/img/resized/ann2.jpeg,../assets/resume/ann.pdf,3
3,Brandes,Moore,"""Every day I wake up is another chance to get ...",My data journey has led me to question everyth...,https://github.com/brandesmoore/,https://www.linkedin.com/in/brandesmoore,True,https://github.com/brandesmoore/capstone_track...,https://youtu.be/a9c24UOM_NU,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/brandes1.jpeg,../assets/img/resized/brandes2.jpeg,../assets/resume/brandes.pdf,4
4,Brant,Ivey,"""People...operate with beliefs and biases. To ...","When someone asks a question, I want to know m...",https://github.com/BrantIvey,https://www.linkedin.com/in/brant-ivey/,True,https://github.com/BrantIvey/NSS_Capstone,https://youtu.be/Qp2ECpwOxi0,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/brant1.jpeg,../assets/img/resized/brant2.jpeg,../assets/resume/brant.pdf,5
5,Christina,Hmun,"""To overcome the insight deficit, Big Data-no ...",,https://github.com/Kris-nge,www.linkedin.com/in/chmun,True,https://github.com/Kris-nge/Nashville-Davidson...,https://youtu.be/pmRBqD-XOz0,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/christina1.jpeg,../assets/img/resized/christina2.jpeg,../assets/resume/christina.pdf,6
6,David,Mellow,"""Facts do not cease to exist because they are ...",I look forward to telling meaningful stories t...,https://github.com/DavidMellow,https://www.linkedin.com/in/davidmellow/,True,https://github.com/DavidMellow/tn-elections,https://youtu.be/6--LquoAusk,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/david1.jpeg,../assets/img/resized/david2.jpeg,../assets/resume/david.pdf,7
7,Elissa,Newland,"""Making the simple complicated is commonplace;...",A manager of creative teams and projects in Si...,https://github.com/ECNewland,https://www.linkedin.com/in/elissacnewland/,False,https://github.com/ECNewland/DAM_Success_Capstone,NaN,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/elissa1.jpeg,../assets/img/resized/elissa2.jpeg,None,8
8,Erin,Anderson,"Ask not whether it can be done, but rather how...",Former litigator with a passion for interpreti...,https://github.com/erinmarieanderson,NaN,False,https://github.com/erinmarieanderson/nss-capstone,https://youtu.be/v7uMLpl0wkM,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/erin1.jpeg,../assets/img/resized/erin2.jpeg,../assets/resume/erin.pdf,9
9,Fitsum,Dumessa,"""Let the dataset change your mindset""\n― Hans ...",I have always loved numbers and solving proble...,https://github.com/fdumessa,http://www.linkedin.com/in/fitsum-dumessa,True,https:

---

Rename the columns to something that can be ingested by the js

In [5]:
column_map = {
    'First Name': 'firstName',
    'Last Name': 'lastName',
    'Tagline': 'reelThemIn',
    'Bio': 'bio',
    'GitHub': 'github',
    'LinkedIn': 'linkedin',
    'Is Job Searching': 'job_searching',
    'Website Portfolio': 'portfolio',
    'Capstone Video': 'capstoneVideo',
    'Podcast iframe': 'podcast'
}

website_info = website_info.rename(column_map, axis=1)
website_info = website_info[column_map.values()]

website_info.columns

Index(['firstName', 'lastName', 'reelThemIn', 'bio', 'github', 'linkedin',
       'job_searching', 'portfolio', 'capstoneVideo', 'podcast'],
      dtype='object')

---

Make the paragraphs

In [6]:
def html_paragraph(v):
    if pd.isna(v):
        return ""
    return str(v)


website_info['reelThemIn'] = website_info['reelThemIn'].apply(html_paragraph)
website_info['bio'] = website_info['bio'].apply(html_paragraph)

---

Clean up some excess characters

In [7]:
for col in website_info.columns:
    try:
        website_info[col] = website_info[col] \
            .str.replace('“', '"') \
            .str.replace('”', '"') \
            .str.replace("’", "'") \
            .str.replace("—", "-")
    except:
        pass

---

Gather the image paths

In [8]:
def image_path(name, idx):
    return f"../assets/img/resized/{name.lower()}{idx}.jpeg"


website_info['proImg'] = website_info['firstName'].apply(image_path, idx=1)
website_info['funImg'] = website_info['firstName'].apply(image_path, idx=2)

In [ ]:
website_info['proImg'].head()

---

Resume paths

In [9]:
def resume_path(name):
    path = f"../assets/resume/{name.lower()}.pdf"
    if os.path.exists(path):
        return path


website_info['resume'] = website_info['firstName'].apply(resume_path)
website_info[['firstName', 'resume']]

,firstName,resume
0,Adam,../assets/resume/adam.pdf
1,Alan,../assets/resume/alan.pdf
2,Ann,../assets/resume/ann.pdf
3,Brandes,../assets/resume/brandes.pdf
4,Brant,../assets/resume/brant.pdf
5,Christina,../assets/resume/christina.pdf
6,David,../assets/resume/david.pdf
7,Elissa,None
8,Erin,../assets/resume/erin.pdf
9,Fitsum,../assets/resume/fitsum.pdf


---

Assign Ids

In [10]:
website_info['id'] = website_info.index + 1

---

Take a peek

In [11]:
website_info.head(1)

,firstName,lastName,reelThemIn,bio,github,linkedin,job_searching,portfolio,capstoneVideo,podcast,proImg,funImg,resume,id
0,Adam,Tsuchiyama,"""In God we trust. All others, bring data."" - W...",Whether it's determining how to incorporate th...,https://github.com/adamtsuch21,https://www.linkedin.com/in/adam-tsuchiyama/,True,https://github.com/adamtsuch21/nss_capstone,https://youtu.be/FB-Ia6q_Yyc,"<iframe width=""100%"" height=""166"" scrolling=""n...",../assets/img/resized/adam1.jpeg,../assets/img/resized/adam2.jpeg,../assets/resume/adam.pdf,1


In [12]:
website_info.to_json("./../data/cohort.json", orient='records')